In [28]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [29]:
data = pd.read_csv("train.csv")
data.head()

,ID,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n4087,n4088,n4089,n4090,n4091,n4092,n4093,n4094,n4095,category
0,0,0.0,0.000000,1.272801,0.290501,0.581446,0.000000,0.000000,0,0.000000,...,1.645888,0.869640,0.302432,0.953719,0.022545,0.498048,0.0,0.034988,0.692382,Orange_Ripe
1,1,0.0,0.000000,1.542096,0.000000,0.896557,0.049978,0.000000,0,0.117847,...,1.504220,0.622686,0.588427,0.524415,0.305426,0.386204,0.0,0.000000,0.668196,Banana_Ripe
2,2,0.0,0.000000,1.098595,0.571866,0.500355,0.000000,0.000000,0,0.493137,...,1.169341,0.913239,0.064404,0.531270,0.000000,0.471604,0.0,0.000000,0.658250,Mango_Raw
3,3,0.0,0.101666,1.159194,0.599216,0.893206,0.000000,0.200139,0,0.645675,...,0.560686,1.243676,0.432523,0.701881,0.000000,0.589985,0.0,0.000000,0.591165,Leeche_Raw
4,4,0.0,0.000000,1.178603,0.362568,0.577602,0.000000,0.000000,0,0.079862,...,1.206032,0.736831,0.345906,0.878515,0.119000,0.261441,0.0,0.000000,0.458905,Mango_Ripe


In [30]:
x = data.iloc[:, 1:-1].values
x = np.array(x, dtype=float)
y = data.iloc[:, -1].values

In [31]:
print(set(y))

{'Mango_Raw', 'Pomengranate_Ripe', 'Guava_Raw', 'Leeche_Ripe', 'Coconut_Ripe', 'Leeche_Raw', 'Orange_Ripe', 'Strawberry_Raw', 'Coconut_Raw', 'Apple_Ripe', 'Banana_Ripe', 'Banana_Raw', 'Mango_Ripe', 'Guava_Ripe', 'Pomengranate_Raw', 'Papaya_Raw', 'Apple_Raw', 'Orange_Raw', 'Strawberry_Ripe', 'Papaya_Ripe'}


In [32]:
#converting strings to numeric labels
label = 0
labels = {}
inverse_labels = {}
for i in set(y):
    labels[i] = label
    inverse_labels[label] = i
    label += 1

for i in range(len(y)):
    y[i] = labels[y[i]]

y = np.array(y, dtype=float)

print(f"Number of classes: {label}")
pd.DataFrame(y).head()

Number of classes: 20


,0
0,6.0
1,10.0
2,0.0
3,5.0
4,12.0


In [33]:
class Clustering(BaseEstimator, TransformerMixin):
    def __init__(self, eps, min_samples):
        self.eps = eps
        self.min_samples = min_samples

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        self.dbscan_ = DBSCAN(eps=self.eps, min_samples=self.min_samples)
        labels = self.dbscan_.fit_predict(X)
        
        return np.concatenate((X, labels.reshape(-1, 1)), axis=1)

In [34]:
class OutlierRemoval(BaseEstimator, TransformerMixin):
    def __init__(self, n_neighbors, contamination):
        self.n_neighbors = n_neighbors
        self.contamination = contamination

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        self.lof = LocalOutlierFactor(n_neighbors=self.n_neighbors, contamination=self.contamination)
        is_inlier = self.lof.fit_predict(X) == 1

        return X[is_inlier], y[is_inlier]

In [35]:
pipe = Pipeline(
    [
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=500)),
    ('lof', OutlierRemoval(n_neighbors=5, contamination=0.1)),
    ('dbscan', Clustering(eps=15, min_samples=10)),
    ('knn', KNeighborsClassifier(n_neighbors=60)),
    ]
)

In [36]:
scores = cross_val_score(pipe, x, y, cv=5)

print(np.mean(scores))

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\rajad\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\memory.py", line 349,